In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.models import Model, save, load_model
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Input, Lambda, Dense, Flatten, Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Unzipping data**

In [ ]:
# !unzip '/content/drive/MyDrive/IEEE/Dataset/Dataset.zip' -d '/content/drive/MyDrive/IEEE/Dataset/'

In [ ]:
# !unzip '/content/drive/MyDrive/IEEE/Dataset/Brighten Down_Train.zip' -d '/content/drive/MyDrive/IEEE/Dataset/Upload/train'

In [ ]:
# !unzip '/content/drive/MyDrive/IEEE/Dataset/Brighten Down_Val.zip' -d '/content/drive/MyDrive/IEEE/Dataset/Upload/validation'

### **Performing Data Augmentation**

In [ ]:
Trainpath = "/content/drive/MyDrive/IEEE/Dataset/train"
Validpath = "/content/drive/MyDrive/IEEE/Dataset/validation"

datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range = 40, 
                    width_shift_range = 0.2, 
                    height_shift_range = 0.2, 
                    shear_range = 0.2)

train_generator = datagen.flow_from_directory(
    Trainpath,
    target_size=(224,224),
    batch_size = 32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    Validpath,
    target_size=(224,224),
    batch_size = 32,
    class_mode='categorical'
)

Found 3200 images belonging to 4 classes.
Found 800 images belonging to 4 classes.


### **Using Transfer Learning**

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

58900480/58889256 [==============================] - 1s 0us/step


### **Defining our own NN layers**

In [ ]:
# Flatten the output layer to 1 dimension
x = Flatten()(base_model.output)

x = Dense(1024, activation='relu')(x)

x = Dense(512, activation='relu')(x)

x = Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = Dense(4, activation='softmax')(x)

model = Model(base_model.input, x)

model.compile(optimizer = Adam(learning_rate=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])

### **Model Training**

In [ ]:
model.fit_generator(train_generator, validation_data = validation_generator, epochs = 25)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/25
100/100 [==============================] - 1005s 10s/step - loss: 0.3052 - acc: 0.8825 - val_loss: 0.0266 - val_acc: 0.9925
Epoch 2/25
100/100 [==============================] - 54s 536ms/step - loss: 0.0340 - acc: 0.9897 - val_loss: 0.0780 - val_acc: 0.9725
Epoch 3/25
100/100 [==============================] - 52s 523ms/step - loss: 0.0214 - acc: 0.9931 - val_loss: 0.0107 - val_acc: 0.9962
Epoch 4/25
100/100 [==============================] - 53s 527ms/step - loss: 0.0150 - acc: 0.9956 - val_loss: 0.0023 - val_acc: 1.0000
Epoch 5/25
100/100 [==============================] - 53s 534ms/step - loss: 0.0098 - acc: 0.9969 - val_loss: 0.0054 - val_acc: 0.9975
Epoch 6/25
100/100 [==============================] - 53s 528ms/step - loss: 0.0076 - acc: 0.9975 - val_loss: 0.0158 - val_acc: 0.9950
Epoch 7/25
100/100 [==============================] - 53s 534ms/step - loss: 0.0028 - acc: 0.9991 - val_loss: 2.9356e-04 - val_acc: 1.0000
Epoch 8/25
100/100 [==============================]

**Saving the model**

In [ ]:
model.save("/content/drive/MyDrive/IEEE/model.h5")